In [11]:
import pandas as pd
import numpy as np
import string
import random
import torch
import collections
import dgl
import tqdm
import sklearn.metrics as metrics
import os
os.environ["DATASET_DIR"] = "/Users/rustamwarwick/Documents/Warwick/d3-gnn/datasets"

In [92]:
class ThreeLayerGraphSAGE(torch.nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.conv1 = dgl.nn.SAGEConv(in_features, hidden_features, aggregator_type="mean")
        self.conv2 = dgl.nn.SAGEConv(hidden_features, hidden_features, aggregator_type="mean")
        self.conv3 = dgl.nn.SAGEConv(hidden_features, out_features, aggregator_type="mean")
        self.ff = torch.nn.Sequential(
            torch.nn.Linear(out_features, out_features),
            torch.nn.ReLU(),
            torch.nn.Linear(out_features, out_features),
            torch.nn.ReLU(),
            torch.nn.Linear(out_features, out_features),
            torch.nn.ReLU(),
        )
    def forward(self, g, x):
        x = self.conv1(g[0], x)
        x = self.conv2(g[1], x)
        x = self.conv3(g[2], x)
        return torch.nn.functional.softmax(self.ff(x), dim=1)


In [125]:
class SAGE(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(dgl.nn.SAGEConv(in_channels, hidden_channels, aggregator_type="mean"))
        self.bns = torch.nn.ModuleList()
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(dgl.nn.SAGEConv(hidden_channels, hidden_channels, aggregator_type="mean"))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(dgl.nn.SAGEConv(hidden_channels, out_channels, aggregator_type="mean"))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, g, x):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(g[i],x)
            if self.batchnorm: 
                x = self.bns[i](x)
            x = torch.relu(x)
            x = torch.dropout(x, p=self.dropout, train=self.training)
        x = self.convs[-1](g[-1],x)
        return x.log_softmax(dim=-1)

# Get OGB-Products

In [37]:
def get_ogb(nedge=20000000):
    topology = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "ogb-products","edges.csv"), header=None, nrows=nedge)
    features = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "ogb-products","node_features.csv"), header=None, index_col=0)
    labels = pd.read_csv(os.path.join(os.environ["DATASET_DIR"], "ogb-products","node_labels.csv"), header=None, index_col=0)
    graph = dgl.graph((topology[0].values, topology[1].values))
    graph.ndata['f'] = torch.tensor(features.values, dtype=torch.float32)
    graph.ndata['l'] = torch.tensor(labels.values[:,0], dtype=torch.int64)
    return graph

def save_ogb_model(model):
    os.mkdir(os.path.join(os.environ["DATASET_DIR"], "ogb-products","graphSage"))
    for i, weight in enumerate(model.parameters()):
        np.save(os.path.join(os.environ["DATASET_DIR"], "ogb-products","graphSage",str(i)), weight.data)
    
def load_ogb_model(model):
    for i, j in enumerate(model.parameters()):
        d = np.load(os.path.join(os.environ["DATASET_DIR"], "ogb-products","graphSage",str(i) + '.npy'))
        j.data = torch.tensor(np.load(os.path.join(os.environ["DATASET_DIR"], "ogb-products","graphSage",str(i) + '.npy')), requires_grad=True)
        
    

# Get GRaphFin

In [88]:
def get_dgraphFin():
    # 0 non-fraud, 1-fraud
    res = np.load(os.path.join(os.environ["DATASET_DIR"], "DGraphFin","dgraphfin.npz"))
    edges = res['edge_index']
    g = dgl.graph((edges[:,0], edges[:,1]))
    g.ndata["x"] = torch.tensor(res['x']).type(torch.float32)
    g.ndata["y"] = torch.tensor(res['y'])
    return g, res['train_mask'], res['test_mask']

In [89]:
g,train_mask, test_mask = get_dgraphFin()

# Train

In [133]:
model = SAGE(17, 128, 2, 3, 0.3)
sampler = dgl.dataloading.MultiLayerNeighborSampler([20,50, 100])
sampler_full = dgl.dataloading.MultiLayerFullNeighborSampler(3)
class_imbalance = (g.ndata['y'] == 1).sum() / (g.ndata['y'] == 0).sum()
dataloader = dgl.dataloading.DataLoader(
    g, train_mask, sampler,
    batch_size=512,
    shuffle=True,
    drop_last=False,
    num_workers=4)
test_dataloader = dgl.dataloading.DataLoader(
    g, test_mask, sampler_full,
    batch_size=test_mask.shape[0],
    shuffle=True,
    drop_last=False,
    num_workers=4)
opt = torch.optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-5)
loss = torch.nn.NLLLoss(weight=torch.tensor([1, 1/class_imbalance]))

In [139]:
for epoch in tqdm.tqdm(range(20)):
    losses = list()
    for input_nodes, output_nodes, blocks in dataloader:
        input_features = blocks[0].srcdata['x']
        output_labels = blocks[-1].dstdata['y']
        output_predictions = model(blocks, input_features)
        loss_value = loss(output_predictions, output_labels)
        opt.zero_grad()
        loss_value.backward()
        losses.append(loss_value.item())
        opt.step()
    print(np.array(losses).sum() / len(losses))

  5%|██████████                                                                                                                                                                                                | 1/20 [00:31<10:00, 31.62s/it]

0.8355583029850286


 10%|████████████████████▏                                                                                                                                                                                     | 2/20 [01:04<09:40, 32.26s/it]

0.7248738753233719


 15%|██████████████████████████████▎                                                                                                                                                                           | 3/20 [01:36<09:04, 32.02s/it]

...

0.5816441583747226


 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 19/20 [09:27<00:28, 28.71s/it]

0.5814817412300725


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [09:55<00:00, 29.80s/it]

0.5815178223292128


In [144]:
model.train(False)
for input_nodes, output_nodes, blocks in test_dataloader:
        input_features = blocks[0].srcdata['x']
        output_labels = blocks[-1].dstdata['y']
        output_predictions = model(blocks, input_features).exp()
        

In [143]:
output_predictions.exp()

tensor([[0.6971, 0.3029],
        [0.5382, 0.4618],
        [0.8581, 0.1419],
        ...,
        [0.8084, 0.1916],
        [0.4573, 0.5427],
        [0.4743, 0.5257]], grad_fn=<ExpBackward0>)

In [145]:
print("Accuracy:",metrics.accuracy_score(output_predictions.argmax(axis=1),output_labels))

print("recall:", metrics.recall_score(output_predictions.argmax(axis=1),output_labels))

print("precision:", metrics.precision_score(output_predictions.argmax(axis=1),output_labels))

print("f1_score:", metrics.f1_score(output_predictions.argmax(axis=1),output_labels))

Accuracy: 0.6778013489991297
recall: 0.026934011671405057
precision: 0.6964746345657782
f1_score: 0.051862404558769386
